In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.integrate import simpson
import ipywidgets as widgets

def eV2J(E_ev):
    return E_ev*1.6e-19

def J2eV(E_J):
    return E_J / 1.6e-19

def k2wl(k):
    return 2*np.pi/k

def wl2k(wl_nm): # for photon
    wl = wl_nm*1e9
    k_em = 2*np.pi/wl
    return k_em

# converts ELECTRON energy in [eV] to wavevector in [1/m]
def e_energy_ev_2k(E):
    return np.sqrt((2*m_e*E)/(hbar**2))

# converts PHOTON energy in [eV] to wavevector in [1/m]
def ph_energy_ev_2k(E):
    return E/(hbar*c)

c = 3.0e8                                    # speed of light [m/s]
k_B = 8.617333262145e-5                      # Boltzmann constant [eV/K]
hbar = 6.582119569e-16                       # Reduced Planck constant [eV*s]
m_e = 5.485e-4                               # electron mass [eV*(s/m)^2]
T_e = 300                                    # K
E_F = 5.0                                    # Chemical potential [eV]
T_F = E_F/k_B                                # Fermi temperature [K]
E_L = 2.25                                   # pump energy [eV]
k_F = e_energy_ev_2k(E_F)                    # Fermi wavevector [1/m]
EF_L = 5e10                                  # pump field strength [V^2/m^2]
EF_sat = 1e13                                # saturation field [V^2/m^2]
delta_E = (abs( EF_L / EF_sat)**2)           # non-thermal distribution factor [a.u.]

k_int_max = ph_energy_ev_2k(10)

In [2]:
# takes wavevector [1/m] returns corresponding electron energy [eV]
def E_e(k_e):
    return ((hbar*k_e)**2)/(2*m_e)

# Fermi-Dirac distribution
def f_T(k_e, E_ph):   
    mu = E_F*( 1 - ((np.pi**2)/12)*((T_e/T_F)**2))      # chemical potential
    E = E_e(k_e) + E_ph - mu                            # total energy
    return 1 / (np.exp( E / (k_B * T_e)) + 1)


# thermal emission integrand
def j_T(k1, k2, E_ph):
    return f_T(k1,E_ph) * (1 - f_T(k2,0)) * 4*np.pi*(k1**2) * 4*np.pi*(k2**2)


# nonthermal distribution
def f_NT(k_e, E_ph):
    A = f_T(k_e, E_ph - E_L)*( 1 - f_T(k_e, E_ph) )
    B = f_T(k_e, E_ph)*( 1 - f_T(k_e, E_ph + E_L) )
    return delta_E*(A-B)


# nonequilibrium distribution
def f(k_e, E_ph):
    return f_T(k_e, E_ph) + f_NT(k_e, E_ph)


# non-equilibrium emission integrand
def j(k_e, E_ph):
    return f(k_e , E_ph) * (1 - f(k_e, 0)) * 4*np.pi*(k_e**2)

# integration constants
N = 10000
k_min = 0
k_max = e_energy_ev_2k(10)
dk = (k_max - k_min) / N

# emission at E_ph
def I_e_T_numeric(E_ph):
    k = np.arange(k_min, k_max, dk)
    return simpson(j_T(k, E_ph), k)

def I_e_numeric(E_ph):
    k = np.arange(k_min, k_max, dk)
    return simpson(j(k, E_ph), k)

k = np.arange(k_min, k_max, dk)

In [3]:
def plot_comparison(E_em_values, I_e_T_numeric_vals,  I_e_T_analytic_vals):

    relative_error_vals = np.abs((I_e_T_analytic_vals - I_e_T_numeric_vals)/(I_e_T_analytic_vals))

    fig = plt.figure(figsize = (6,8), dpi = 100)

    # Create a GridSpec with 5 rows and 1 column
    gs = gridspec.GridSpec(5, 1, height_ratios=[4, 4, 4, 4, 4])

    ax2 = plt.subplot(gs[2:5, 0])
    ax1 = plt.subplot(gs[1, 0], sharex=ax2)
    ax1.label_outer()

    # ax1.set_title(r"thermal electronic contribution to emission (with eDOS)", fontsize = 12)
    ax1.plot(E_em_values, np.log10(relative_error_vals), label=r"$log_{10}(\delta_{rel})$",color = 'k')
    ax1.legend(frameon=False, fontsize = 11)
    # ax1.set_yticks([-3,-2,-1,0])

    ax2.semilogy(E_em_values, I_e_T_numeric_vals, label=r"numeric", linewidth = 6)
    ax2.semilogy(E_em_values, I_e_T_analytic_vals, linestyle = "--", label = r"analytic", linewidth = 2)

    ax2.set_xlabel('$\\omega / \\omega_{ L}$', fontsize = 14)
    ax2.set_ylabel(r'$I_e$', fontsize = 16)

    ax2.set_xlim(min(E_em_values), max(E_em_values))

    x_ticks = np.array([0, E_L, E_F, 8])
    x_tick_labels = np.round( (1/E_L)*np.array([0, E_L, E_F, max(x_ticks)]) , 2)
    ax2.set_xticks(x_ticks, x_tick_labels)

    ax2.legend(frameon=False, fontsize = 11)

In [4]:
# density of states
def g(E):
    return 4*np.pi*( ((m_e)**(3/2))/(hbar**3) )*np.sqrt(2*E)

# black body energy
def E_BB(E_ph):
    return E_ph / (np.exp((E_ph) / (k_B * T_e)) - 1)

# analytic emission
def I_e_T_analytic(E_ph):
        return E_BB(E_ph) * g(E_F) * g(E_F)

   
def I_e_analytic(E_ph):
    A = 2*E_BB(E_ph - E_L)
    B = -2*E_BB(E_ph - E_L) + E_BB(E_ph - 2*E_L)
    
    return ( E_BB(E_ph) + delta_E*A + delta_E*delta_E*B ) * g(E_F) * g(E_F)


E_ph_vals = np.linspace(0.01, 8.0, 100)
I_e_T_numeric_vals = np.array( [I_e_T_numeric(E_ph) for E_ph in E_ph_vals] )
I_e_T_analytic_vals = np.array( I_e_T_analytic(E_ph_vals) )

I_e_numeric_vals = np.array( [I_e_numeric(E_ph) for E_ph in E_ph_vals] )
I_e_analytic_vals = np.array( I_e_analytic(E_ph_vals) )



plot_comparison(E_ph_vals, I_e_T_numeric_vals, I_e_T_analytic_vals)
# plot_comparison(E_ph_vals, I_e_numeric_vals, I_e_analytic_vals)

TypeError: j_T() missing 1 required positional argument: 'E_ph'

In [ ]:
plt.plot(E_ph_vals, np.log10(I_e_T_numeric_vals))
plt.plot(E_ph_vals, np.log10(I_e_numeric_vals))
# plt.plot(E_ph_vals, np.log10(I_e_numeric_vals - I_e_T_numeric_vals))

In [ ]:
T_e = 5000
E_ph_valss = np.linspace(0,10,1000)
I_e_T_numeric_valss = np.array( [(E_ph**2)*I_e_T_numeric(E_ph) for E_ph in E_ph_valss] )
plt.plot(E_ph_valss, I_e_T_numeric_valss)